In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import bz2
import re
import os
import gc
import matplotlib.pyplot as plt

In [ ]:
train_file = bz2.BZ2File('train.ft.txt.bz2')
test_file = bz2.BZ2File('test.ft.txt.bz2')

In [ ]:
def load_extract(file):
    texts, labels = [], [] 
    for line in file:
        x = line.decode('utf-8')  # decode binary to string
        labels.append(int(x[9]) - 1)  # extract labels
        texts.append(x[10:].strip())  # extract texts
    print('Done !')
    return np.array(labels), texts

In [ ]:
train_labels, train_texts = load_extract(train_file)
test_labels, test_texts = load_extract(test_file)

Done !
Done !


In [ ]:
train_data = pd.DataFrame(data=train_labels, columns=['label'])
train_data['text'] = train_texts

In [ ]:
train_data.head()

,label,text
0,1,Stuning even for the non-gamer: This sound tra...
1,1,The best soundtrack ever to anything.: I'm rea...
2,1,Amazing!: This soundtrack is my favorite music...
3,1,Excellent Soundtrack: I truly like this soundt...
4,1,"Remember, Pull Your Jaw Off The Floor After He..."


In [ ]:

test_data = pd.DataFrame(data=test_labels, columns=['label'])
test_data['text'] = test_texts

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

2023-04-15 13:50:39.844606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel7/java/1.8.0_131/jre/lib/amd64/server/:/N/soft/rhel7/curl/intel/7.54.0/lib:/N/soft/rhel7/python/3.9.8/lib:/N/soft/rhel7/openmpi/gnu/4.0.4/lib:/N/soft/rhel7/libpng/1.2.57/lib:/N/soft/rhel7/gcc/9.3.0/lib64:/N/soft/rhel7/gcc/9.3.0/lib:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/ipp/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/mkl/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/tbb/lib/intel64/gcc4.7:/N/soft/rhel7/intel/19.5/debugger_2019/iga/lib:/N/soft/rhel7/intel/19.5/d

In [ ]:
sample_train = train_data.sample(n=1000000, random_state=2)


In [ ]:
sample_test = test_data.sample(n=1000000, random_state=2)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
sample_train.head()

,label,text
3281328,1,Excellent home help for parents: Volume 1 of D...
2662721,0,"Stay far, far away.: I made it through about 6..."
1600544,0,Lost Woods: Lost WoodsI didn't really understa...
815246,0,"Renaissance -12, Women's brown suead shoes: Re..."
1254178,1,Best Novel I've Read This Year: Intrigued by t...


In [ ]:
text = sample_train['text'].values
label = sample_train['label'].values

In [ ]:
text_test = sample_test['text'].values
label_test = sample_test['label'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(text, label, test_size=0.2, random_state=42)


array([1, 1, 0, ..., 0, 1, 1])

In [ ]:
#training data
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)


In [ ]:
#testing data
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_val)

In [ ]:
#train data
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=50, padding='post', truncating='post')

In [ ]:
#test data
X_test_sequences = tokenizer.texts_to_sequences(X_val)
X_test_padded = pad_sequences(X_test_sequences, maxlen=50, padding='post', truncating='post')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=50),
    LSTM(64),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
# Train the model and see the loss and accuracy on each epoch
history = model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_val), epochs=10, batch_size=32, verbose=1)

# Print the history object containing the loss and accuracy on each epoch
print(history.history)


Epoch 1/10
313/313 [==============================] - 18s 44ms/step - loss: 0.4858 - accuracy: 0.7579 - val_loss: 1.0421 - val_accuracy: 0.5559
Epoch 2/10
313/313 [==============================] - 13s 40ms/step - loss: 0.2832 - accuracy: 0.8923 - val_loss: 1.2580 - val_accuracy: 0.5411
Epoch 3/10
313/313 [==============================] - 12s 40ms/step - loss: 0.2187 - accuracy: 0.9184 - val_loss: 1.0782 - val_accuracy: 0.5364
Epoch 4/10
313/313 [==============================] - 12s 40ms/step - loss: 0.1645 - accuracy: 0.9403 - val_loss: 1.2429 - val_accuracy: 0.5497
Epoch 5/10
313/313 [==============================] - 12s 40ms/step - loss: 0.1218 - accuracy: 0.9580 - val_loss: 1.7287 - val_accuracy: 0.5450
Epoch 6/10
313/313 [==============================] - 12s 39ms/step - loss: 0.1047 - accuracy: 0.9646 - val_loss: 1.8973 - val_accuracy: 0.5508
Epoch 7/10
313/313 [==============================] - 13s 40ms/step - loss: 0.0965 - accuracy: 0.9684 - val_loss: 1.5139 - val_accuracy:

In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model.evaluate(X_test_padded, label_test)

# Print the validation accuracy
print('Validation accuracy:', val_acc)


313/313 [==============================] - 3s 10ms/step - loss: 1.9885 - accuracy: 0.5520
Validation accuracy: 0.5519999861717224


In [ ]:
# Train the model and see the loss and accuracy on each epoch
history = model.fit(X_train_padded, label, validation_data=(X_test_padded, label_test), epochs=50, batch_size=32, verbose=1)

# Print the history object containing the loss and accuracy on each epoch
print(history.history)

Epoch 1/50
3125/3125 [==============================] - 109s 34ms/step - loss: 0.3357 - accuracy: 0.8563 - val_loss: 0.2736 - val_accuracy: 0.8875
Epoch 2/50
3125/3125 [==============================] - 105s 34ms/step - loss: 0.2481 - accuracy: 0.8987 - val_loss: 0.2622 - val_accuracy: 0.8928
Epoch 3/50
3125/3125 [==============================] - 106s 34ms/step - loss: 0.2133 - accuracy: 0.9150 - val_loss: 0.2580 - val_accuracy: 0.8942
Epoch 4/50
3125/3125 [==============================] - 105s 34ms/step - loss: 0.1839 - accuracy: 0.9281 - val_loss: 0.2624 - val_accuracy: 0.8930
Epoch 5/50
3125/3125 [==============================] - 105s 34ms/step - loss: 0.1587 - accuracy: 0.9392 - val_loss: 0.3076 - val_accuracy: 0.8912
Epoch 6/50
3125/3125 [==============================] - 105s 34ms/step - loss: 0.1359 - accuracy: 0.9490 - val_loss: 0.3075 - val_accuracy: 0.8878
Epoch 7/50
3125/3125 [==============================] - 104s 33ms/step - loss: 0.1150 - accuracy: 0.9580 - val_loss: 0

In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model.evaluate(X_test_padded, label_test)

# Print the validation accuracy
print('Validation accuracy:', val_acc)

3125/3125 [==============================] - 31s 10ms/step - loss: 0.8761 - accuracy: 0.8758
Validation accuracy: 0.875819981098175


In [ ]:
# Train the model and see the loss and accuracy on each epoch
history = model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_val), epochs=25, batch_size=32, verbose=1)

# Print the history object containing the loss and accuracy on each epoch
print(history.history)


Epoch 1/25
25000/25000 [==============================] - 679s 27ms/step - loss: 0.2427 - accuracy: 0.9003 - val_loss: 0.2085 - val_accuracy: 0.9161
Epoch 2/25
25000/25000 [==============================] - 666s 27ms/step - loss: 0.1969 - accuracy: 0.9220 - val_loss: 0.1946 - val_accuracy: 0.9220
Epoch 3/25
25000/25000 [==============================] - 668s 27ms/step - loss: 0.1797 - accuracy: 0.9296 - val_loss: 0.1919 - val_accuracy: 0.9242
Epoch 4/25
25000/25000 [==============================] - 671s 27ms/step - loss: 0.1664 - accuracy: 0.9356 - val_loss: 0.1925 - val_accuracy: 0.9244
Epoch 5/25
25000/25000 [==============================] - 669s 27ms/step - loss: 0.1547 - accuracy: 0.9404 - val_loss: 0.1959 - val_accuracy: 0.9239
Epoch 6/25
25000/25000 [==============================] - 669s 27ms/step - loss: 0.1445 - accuracy: 0.9452 - val_loss: 0.2022 - val_accuracy: 0.9243
Epoch 7/25
25000/25000 [==============================] - 675s 27ms/step - loss: 0.1351 - accuracy: 0.9490

In [ ]:
val_loss, val_acc = model.evaluate(X_test_padded, y_val)

# Print the validation accuracy
print('Validation accuracy:', val_acc)

6250/6250 [==============================] - 60s 10ms/step - loss: 0.2725 - accuracy: 0.9148
Validation accuracy: 0.9147650003433228


In [ ]:
# assuming 'model' is your trained Keras model object
model.save('sentiment_analysis_model.h5')


In [ ]:
# assuming 'model' is your trained Keras model object

# Preprocess the input sentence
input_sentence = "Wow. This is great!"
# input_sentence = preprocess_text(input_sentence)

# Convert the input sentence to a numerical format
input_sequence = tokenizer.texts_to_sequences([input_sentence])
input_sequence = pad_sequences(input_sequence, maxlen=50)

# Make predictions using the model
predicted_sentiment = model.predict(input_sequence)
print(predicted_sentiment)

# Print the predicted sentiment
if predicted_sentiment > 0.5:
    print("Positive sentiment")
else:
    print("Negative sentiment")


[[0.88651156]]
Positive sentiment
